In [1]:
import pandas as pd
import sqlite3

create a connection to the database using the library sqlite3


In [2]:
con = sqlite3.connect("../data/checking-logs.sqlite.sqlite_copy")

using only one query for each of the groups, create two dataframes: 
test_results
and control_results with the columns time and avg_diff and only two rows
    
    ◦ time should have the values: after and before
    
    ◦ avg_diff contains the average delta among all the users for the time period
    before each of them made their first visit to the page and afterward
    
    ◦ only take into account the users that have observations before and after
    
    • we still are not using the lab ’project1’

In [7]:
query = '''
SELECT time,
       avg(diff) AS avg_diff
FROM (SELECT uid,
             CAST( (JulianDay(datetime(deadlines.deadlines, 'unixepoch')) -
                    JulianDay(test.first_commit_ts)
                    ) * 24 as Integer
                  ) as diff,
             CASE WHEN test.first_commit_ts < test.first_view_ts THEN 'before'
             ELSE 'after' END AS time
       FROM test 
       LEFT JOIN deadlines on test.labname = deadlines.labs
       WHERE labname <> 'project1'
      )
WHERE uid in (SELECT uid
              FROM (SELECT uid,
                           CASE WHEN test.first_commit_ts < test.first_view_ts THEN 'before'
                           ELSE 'after' END as time
                    FROM test
                    LEFT JOIN deadlines ON test.labname=deadlines.labs
                    WHERE labname <> 'project1'
                    )
               GROUP BY uid
               HAVING COUNT(DISTINCT time)=2
               )
GROUP BY time
'''
test_results = pd.io.sql.read_sql(query, con)
test_results

,time,avg_diff
0,after,104.6000
1,before,60.5625


In [8]:
query = '''
SELECT time,
       avg(diff) AS avg_diff
FROM (SELECT uid,
             CAST( (JulianDay(datetime(deadlines.deadlines, 'unixepoch')) -
                    JulianDay(control.first_commit_ts)
                    ) * 24 as Integer
                  ) as diff,
             CASE WHEN control.first_commit_ts < control.first_view_ts THEN 'before'
             ELSE 'after'
             END AS time
             FROM control
             LEFT JOIN deadlines ON control.labname=deadlines.labs
             WHERE labname <> 'project1'
      )
WHERE uid IN (SELECT uid
              FROM (SELECT uid,
                           CASE WHEN control.first_commit_ts < control.first_view_ts THEN 'before'
                           ELSE 'after'
                           END AS time
                    FROM control
                    LEFT JOIN deadlines ON control.labname=deadlines.labs
                    WHERE labname <> 'project1'
                    )
              GROUP BY uid
              HAVING COUNT(distinct time)=2
              )
GROUP BY time
'''
control_results = pd.io.sql.read_sql(query, con)
control_results

,time,avg_diff
0,after,117.636364
1,before,99.464286


close the connection

In [11]:
con.close()